In [40]:
import pandas as pd
import numpy as np
from IKSSW import IKSSW
from sklearn.ensemble import RandomForestClassifier

In [41]:
winsize = 400
stream = pd.read_csv("Datasets/2CDT.txt", header=None, index_col=False,sep=',', names=['0', '1', '2'])
stream

,0,1,2
0,-2.173800,-0.30919,1
1,-0.676320,1.07960,1
2,3.939200,2.07350,2
3,0.865890,1.83250,1
4,0.812800,0.51703,1
...,...,...,...
15995,-0.040798,-0.51266,1
15996,1.958700,1.39020,1
15997,4.399500,5.91600,2
15998,4.364600,3.11840,1


In [42]:
stream['2'].replace(1, int(0), inplace=True)
stream['2'].replace(2, int(1), inplace=True)
stream

,0,1,2
0,-2.173800,-0.30919,0
1,-0.676320,1.07960,0
2,3.939200,2.07350,1
3,0.865890,1.83250,0
4,0.812800,0.51703,0
...,...,...,...
15995,-0.040798,-0.51266,0
15996,1.958700,1.39020,0
15997,4.399500,5.91600,1
15998,4.364600,3.11840,0


In [43]:
start = stream.iloc[:winsize, :]
labels = stream.iloc[winsize+1:, -1]
start

,0,1,2
0,-2.17380,-0.30919,0
1,-0.67632,1.07960,0
2,3.93920,2.07350,1
3,0.86589,1.83250,0
4,0.81280,0.51703,0
...,...,...,...
395,-1.56420,-1.43390,0
396,3.00770,-0.03437,0
397,4.22860,4.38970,1
398,1.28150,-4.08470,0


In [44]:
X_train = start.iloc[:, :-1]
X_test = start.iloc[:, -1]

clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, X_test)

RandomForestClassifier(n_estimators=200)

In [45]:
def fit_reference(reference, clf):
    X_train = reference.iloc[:, :-1]
    X_test = reference.iloc[:, -1]
    
    clf = RandomForestClassifier(n_estimators=200)
    clf.fit(X_train, X_test)

In [46]:
window = stream.copy()
window = window.drop('2', axis=1)
window

,0,1
0,-2.173800,-0.30919
1,-0.676320,1.07960
2,3.939200,2.07350
3,0.865890,1.83250
4,0.812800,0.51703
...,...,...
15995,-0.040798,-0.51266
15996,1.958700,1.39020
15997,4.399500,5.91600
15998,4.364600,3.11840


In [68]:
print(start['0'].tolist())

[-2.1738, -0.67632, 3.9392, 0.86589, 0.8128, 1.4876, 3.2044, 3.0159, 3.2515, -0.041503, 3.1595, 2.5003, -0.54471, 0.17047, 3.6292, 3.103, 6.8677, 1.8863, 3.7605, 3.0305, 5.9482, 3.4493, 3.0514, -1.1803, -0.13847, 3.8036, 0.74812, -2.0882, 2.7529, 3.811, 3.9859, -0.99532, -1.2387, 3.9284, 2.6343, 4.0642, 3.0825, 1.0047, 4.2336, 4.3693, -2.151, 3.8747, 4.3343, 3.0016, 2.8601, 4.9485, 2.4319, 2.897, -0.18123, 5.4776, 3.5294, 2.7023, 2.708, 4.5727, 2.3632, 1.4355, 1.9286, 1.6244, 6.5178, -0.76676, -0.72577, 2.87, 1.3512, 4.7265, -3.3553, 2.0434, 3.5642, -2.4383, 7.0937, -1.1263, 0.60938, -1.4991, -0.99012, 5.1167, -0.26662, 5.5202, -4.4009, 0.29438, 3.2368, 5.3865, -0.11288, 2.5919, 1.132, 5.9542, -0.78056, 3.2852, 0.72104, -1.6443, -0.018732, 6.7671, 6.0063, 0.70828, 4.3883, -1.9517, 2.175, 1.8408, 0.27663, -1.3728, 5.1214, 1.3058, 2.0487, 4.5402, 2.5663, 1.007, 0.96807, 1.5761, -1.8846, 3.1428, -0.84498, 4.4723, -0.1413, 0.74564, -0.33468, 3.3744, 5.0827, 5.4566, 2.7526, 4.1475, 4.5741, 

In [67]:
iks = IKSSW(start['0'].tolist())
predicted_values = []
drift_count = 0
for obs in range(winsize+1, len(stream)):
    s = obs-winsize
    predicted_values.append(clf.predict([window.iloc[obs, :]]))
    iks.Increment(window.iloc[obs, 0].tolist())
    if iks.KS:
        print(obs)
        reference = stream.iloc[s:obs, :]
        fit_reference(reference, clf)
        iks = IKSSW(reference)
        drift_count += 1


D:\Anaconda\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


401


D:\Anaconda\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


TypeError: '<=' not supported between instances of 'float' and 'str'